# Exercise: Performance Optimization 1

Optimize the following code.

(The type and size of the input is fixed/may not be changed.)

In [1]:
function work!(A, N)
    D = zeros(N,N)
    for i in 1:N
        D = b[i]*c*A
        b[i] = sum(D)
    end
end

N = 100
A = rand(N,N)
b = rand(N)
c = 1.23

work!(A,N)

In [2]:
using BenchmarkTools
@btime work!($A, $N);

  3.885 ms (402 allocations: 7.71 MiB)


## Optimizations

### Avoiding globals

In [3]:
@code_warntype work!(A,N)

MethodInstance for work!(::Matrix{Float64}, ::Int64)
  from work!(A, N) in Main at In[1]:1
Arguments
  #self#::Core.Const(work!)
  A::Matrix{Float64}
  N::Int64
Locals
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  D::Any
  i::Int64
Body::Nothing
1 ─       (D = Main.zeros(N, N))
│   %2  = (1:N)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = Base.getindex(Main.b, i)::Any
│         (D = %10 * Main.c * A)
│   %12 = Main.sum(D)::Any
│         Base.setindex!(Main.b, %12, i)
│         (@_4 = Base.iterate(%2, %9))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing



In [4]:
function work1!(A, N, b, c) # b and c are now function arguments
    D = zeros(N,N)
    for i in 1:N
        D = b[i]*c*A
        b[i] = sum(D)
    end
end

work1! (generic function with 1 method)

In [5]:
@code_warntype work1!(A,N,b,c)

MethodInstance for work1!(::Matrix{Float64}, ::Int64, ::Vector{Float64}, ::Float64)
  from work1!(A, N, b, c) in Main at In[4]:1
Arguments
  #self#::Core.Const(work1!)
  A::Matrix{Float64}
  N::Int64
  b::Vector{Float64}
  c::Float64
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  D::Matrix{Float64}
  i::Int64
Body::Nothing
1 ─       (D = Main.zeros(N, N))
│   %2  = (1:N)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_6 = Base.iterate(%2))
│   %4  = (@_6 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_6::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = Base.getindex(b, i)::Float64
│         (D = %10 * c * A)
│   %12 = Main.sum(D)::Float64
│         Base.setindex!(b, %12, i)
│         (@_6 = Base.iterate(%2, %9))
│   %15 = (@_6 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing



In [6]:
@btime work1!($A, $N, $b, $c);

  3.831 ms (202 allocations: 7.71 MiB)


### Avoiding globals + temporary allocations

In [7]:
function work2!(A, N, b)
    D = zeros(N,N)
    for i in 1:N
        @. D = b[i]*c*A
        b[i] = sum(D)
    end
end

@btime work2!($A, $N, $b);

  469.669 μs (302 allocations: 85.98 KiB)


In [8]:
function work3!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        @inbounds for j in eachindex(D)
            D[j] = b[i]*c*A[j]
        end
        b[i] = sum(D)
    end
end

@btime work3!($A, $N, $b, $c);

  395.802 μs (2 allocations: 78.17 KiB)


### Avoiding globals + temporary allocations and merging `sum` with loop

In [9]:
function work4!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        s = zero(eltype(D))
        @inbounds @simd for j in eachindex(D)
            D[j] = b[i]*c*A[j]
            s += D[j]
        end
        b[i] = s
    end
end

@btime work4!($A, $N, $b, $c);

  251.117 μs (2 allocations: 78.17 KiB)


### Realizing that one can factor out `b` and `c`

In [10]:
# function work!(A, N)
#     D = zeros(N,N)
#     for i in 1:N
#         D = b[i]*c*A
#         b[i] = sum(D)
#     end
# end

# function work!(A, N)
#     for i in 1:N
#         b[i] = sum(b[i]*c*A)
#     end
# end

# function work!(A, N)
#     for i in 1:N
#         b[i] = b[i]*c*sum(A)
#     end
# end

# function work!(A, N)
#     D = c*sum(A)
#     for i in 1:N
#         b[i] *= D
#     end
# end

function work5!(A, N, b, c)
    D = c * sum(A)
    b .*= D
end

@btime work5!($A, $N, $b, $c);

  1.490 μs (0 allocations: 0 bytes)


In [ ]:
@. x = x * x # -> x. = x .* x

In [ ]:
tmp
for i in eachindex(x)
     = x[i] * x[i]
end
x = tmp